In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from opencage.geocoder import OpenCageGeocode
from geopy import distance

# This notebook finds the distance in miles between the home team's city and visiting team's city. It uses the API OpenCageGeocode

In [3]:
nba = pd.read_csv('nba_split_temp_miles.csv')

In [16]:
nba.columns

Index(['date', 'start_time', 'vis_team', 'home_team', 'vis_points',
       'home_points', 'ot', 'attendance', 'home_initial', 'vis_initial',
       'home_players', 'home_bench', 'vis_players', 'vis_bench', 'referees',
       'score_starter_home', 'score_bench_home', 'score_starter_vis',
       'score_bench_vis', 'ref1', 'ref2', 'ref3', 'vp1', 'vp2', 'vp3', 'vp4',
       'vp5', 'hp1', 'hp2', 'hp3', 'hp4', 'hp5', 'vis_city', 'home_city',
       'vis_travel_mi', 'home_travel_mi', 'home_state', 'home_temp',
       'travel_miles'],
      dtype='object')

In [20]:
nba['travel_miles'] = 0.
miles = nba['travel_miles'].copy().tolist()

In [15]:
nba.iloc[0].home_city +', '+ nba.iloc[0].home_state #testing 

'Toronto, ON'

In [19]:
#A city state dictionary for each city
states = ['ON','CA','NC','IA','FL','NY','FL','PA','TX','TX','UT','AZ','OR','MI','TX','CA','MA',
         'TN','LA','OK','CO','CA','WI','GA','NY','IL','OH','MN','DC']
citystate = {nba.home_city.unique()[i]: states[i] for i in range(len(nba.home_city.unique()))} 
citystate


'New Orleans'

# The below function uses my API key to find the lat and long of cities A and B and then finds the distance between the two in miles

In [7]:
def find_distance(A,B):
    key = 'ab7aad6baffe479aa6dc8c92cb981ea9'  # get api key from:  https://opencagedata.com
    geocoder = OpenCageGeocode(key)
    
    result_A = geocoder.geocode(A)
    lat_A = result_A[0]['geometry']['lat']
    lng_A = result_A[0]['geometry']['lng']
    
    result_B = geocoder.geocode(B)
    lat_B = result_B[0]['geometry']['lat']
    lng_B = result_B[0]['geometry']['lng']  
    
    return (np.round(distance.distance((lat_A,lng_A), (lat_B,lng_B)).miles,decimals=1))

In [12]:
find_distance('Toronto, ON','Portland, OR') #Test

2114.0

# OK that works now lets do every game in a for loop

In [21]:
for i in np.arange(len(nba)):
    city1 = nba.iloc[i].home_city +', '+ citystate[nba.iloc[i].home_city]
    city2 = nba.iloc[i].vis_city +', '+citystate[nba.iloc[i].vis_city]
    if city1==city2:
        d=0
    else:
        d = find_distance(city1,city2)
    nba.iloc[i].travel_miles = d
    miles.append(d)

/Users/kessler.363/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [27]:
miles = miles[971:] # a mistake on my part where I forgot to create an empty list

In [29]:
nba.travel_miles = miles #add as a column 

In [32]:
nba[:10]

,date,start_time,vis_team,home_team,vis_points,home_points,ot,attendance,home_initial,vis_initial,...,hp3,hp4,hp5,vis_city,home_city,vis_travel_mi,home_travel_mi,home_state,home_temp,travel_miles
0,2019-10-22,20:00:00,New Orleans Pelicans,Toronto Raptors,122,130,1,20787,TOR,NOP,...,Pascal Siakam,OG Anunoby,Marc Gasol,New Orleans,Toronto,0,1113,ON,57.10,1113.2
1,2019-10-22,22:30:00,Los Angeles Lakers,Los Angeles Clippers,102,112,0,19068,LAC,LAL,...,Landry Shamet,Patrick Patterson,Ivica Zubac,Los Angeles,Los Angeles,0,0,CA,76.54,0.0
2,2019-10-23,19:00:00,Chicago Bulls,Charlotte Hornets,125,126,0,15424,CHO,CHI,...,Dwayne Bacon,Miles Bridges,Terry Rozier,Chicago,Charlotte,587,0,NC,57.13,587.1
3,2019-10-23,19:00:00,Detroit Pistons,Indiana Pacers,119,110,0,17923,IND,DET,...,Domantas Sabonis,T.J. Warren,Jeremy Lamb,Detroit,Indianapolis,0,240,IA,42.54,488.3
4,2019-10-23,19:00:00,Cleveland Cavaliers,Orlando Magic,85,94,0,18846,ORL,CLE,...,Evan Fournier,D.J. Augustin,Jonathan Isaac,Cleveland,Orlando,0,893,FL,76.45,893.8
5,2019-10-23,19:30:00,Minnesota Timberwolves,Brooklyn Nets,127,126,1,17732,BRK,MIN,...,Kyrie Irving,Caris LeVert,Jarrett Allen,Minneapolis,Brooklyn,1023,0,NY,49.76,1023.2
6,2019-10-23,19:30:00,Memphis Grizzlies,Miami Heat,101,120,0,19600,MIA,MEM,...,Kendrick Nunn,Bam Adebayo,Meyers Leonard,Memphis,Miami,0,872,FL,83.83,872.5
7,2019-10-23,19:30:00,Boston Celtics,Philadelphia 76ers,93,107,0,20422,PHI,BOS,...,Josh Richardson,Al Horford,Joel Embiid,Boston,Philadelphia,0,271,PA,59.28,271.0
8,2019-10-23,20:30:00,Washington Wizards,Dallas Mavericks,100,108,0,19816,DAL,WAS,...,Maxi Kleber,Delon Wright,Courtney Lee,Washington,Dallas,1183,0,TX,68.63,1183.8
9,2019-10-23,20:30:00,New York Knicks,San Antonio Spurs,111,120,0,18354,SAS,NYK,...,DeMar DeRozan,Trey Lyles,Dejounte Murray,New York,San Antonio,1583,0,TX,70.38,1583.2


In [62]:
nba.to_csv('nba_split_temp_miles.csv',index=False) #write to csv